In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import sncosmo
from scipy import signal
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from sklearn.gaussian_process.kernels import WhiteKernel, Matern, RBF, RationalQuadratic, ExpSineSquared
from sklearn.gaussian_process import GaussianProcessRegressor
from matplotlib.ticker import ScalarFormatter
from scipy.interpolate import InterpolatedUnivariateSpline as Spline1d
from tqdm import tqdm
import pickle
import bz2
from ipywidgets import widgets
import PIL

sns.set_style("whitegrid", {'axes.grid':'False','xtick.bottom': True,'ytick.left': True})

params = {"text.usetex" : True,'font.size': 20, 'font.family':'serif', 'font.serif':'Computer Modern'}
plt.rcParams.update(params)


In [2]:
p_grid=np.arange(-10,51,1)         # phase (days from maximum, in the SN rest frame) p=(t_obs-t_max)/(1+z)
w_grid1=np.arange(3350,8710,10)    # wavelength in the SN rest frame lambda= lambda_obs/(1+z)
w_grid2=np.arange(3400,8410,10)

In [3]:
class ScalarFormatterClass1(ScalarFormatter):
    def _set_format(self):
        self.format = "%1.1f"

## Applying filter in SN Factoty SNe Spectras

In [4]:
file_names=glob.glob('./data/sp/*.dat')   # unit: W/m^2/nm times offset, equivalently 10^2 erg/s/cm^2/Angstrom times offset

In [5]:
catalog = pd.read_csv('./data/list.dat', names=['name', 'phase', 'path'], sep='\s+')

In [6]:
sne_list=catalog["name"].unique() # The choice of test sn ou train sn is from SNEMO, we stil havent made this separation

We save the figures with the superposition of actual spectra and filtered spectra. And also save 3d figures of the disposition of said spectras in time.

In [7]:
filtered_reg_data={}

In [8]:
# Discutir: Test_SN7, Test_SN8, Train_SN31, Train_SN30

In [9]:
rmv_specs={"Test_SN2":[25.853,27.738,32.374],
           "Train_SN5":[20.635],
           "Test_SN12":[45.809],
           "Train_SN59":[37.662]}

In [10]:
for sn in tqdm(sne_list):
    pp = PdfPages(f"./plots_filtered_spectras/{sn}.pdf")
    spec_list=catalog[catalog["name"]==sn].reset_index(drop=True)
    filtered_reg_data[sn]={}
    for i in range(len(spec_list)):
        spec_data=sncosmo.read_lc("./data/sp/"+spec_list["path"][i], format='salt2')
        if (sn in rmv_specs.keys()) and (spec_list["phase"][i] in rmv_specs[sn]):
            continue

        fig,ax=plt.subplots(figsize=(10,8))
        # Applying filters
        nyquist_frequecy=1/25 # Alterei de 1/10 para 1/25 porque o fluxo filtrado estava muito suave, pulando varios detalhes
        cutoff=1/100
        
        b, a = signal.butter(8, cutoff/nyquist_frequecy, analog=False)   
        filtered_flux=signal.filtfilt(b, a, spec_data["SN_SPEC"], padlen=0)
        #err_filtered_flux_up=signal.filtfilt(b, a, spec_data["SN_SPEC"]+spec_data["SN_ERR"], padlen=0) # Forma aproximada de levar o erro do espectro em conta
        #err_filtered_flux_down=signal.filtfilt(b, a, spec_data["SN_SPEC"]-spec_data["SN_ERR"], padlen=0)
        cont_error_filtering=(filtered_flux-spec_data["SN_SPEC"])/3  # Nova abordagem para levar em conta o erro na estimativa do espectro filtrado
        error_estimative=np.sqrt(cont_error_filtering**2+spec_data["SN_ERR"]**2)

        eff_grid=w_grid1[(w_grid1>=spec_data["WAVE"][0]) & (w_grid1<=spec_data["WAVE"][-1]-100)]
        
        reg_filt_flux=interp1d(spec_data["WAVE"].value,filtered_flux)(eff_grid)
        error_flux=interp1d(spec_data["WAVE"].value,error_estimative)(eff_grid)
        #err_reg_filt_flux_up=interp1d(spec_data["WAVE"].value,err_filtered_flux_up)(eff_grid)
        #err_reg_filt_flux_down=interp1d(spec_data["WAVE"].value,err_filtered_flux_down)(eff_grid)
        #fluxerr=err_reg_filt_flux_up-reg_filt_flux

        filtered_reg_data[sn][(spec_list["phase"][i])]=pd.DataFrame({"wave":eff_grid,"flux":reg_filt_flux,"err_flux":error_flux})
        
        ax.plot(spec_data["WAVE"],spec_data["SN_SPEC"],label="Spectra at "+str(spec_list["phase"][i])+" days",lw=2,c="#1f77b4",zorder=1)
        ax.fill_between(spec_data["WAVE"],spec_data["SN_SPEC"]+2*spec_data["SN_ERR"],spec_data["SN_SPEC"]-2*spec_data["SN_ERR"],color="#1f77b4",alpha=0.7,zorder=2,label=r"95\% confidence")
        ax.plot(eff_grid,reg_filt_flux,zorder=3,color="#ff7f0e",label="Filtered spectra",lw=2)
        ax.fill_between(eff_grid,reg_filt_flux+2*error_flux,reg_filt_flux-2*error_flux,alpha=0.7,zorder=4,color="#ff7f0e",label=r"95\% confidence")
        ax.set_xlabel(r"Wavelength ($\AA$)")
        ax.set_ylabel(r"Flux (erg/s/cm$^2/\AA$ $\times$ offset)") # não substitui o offset por 10^2, pq no mesmo arquivo que tem a unidade
        ax.legend(fontsize=20)                                    # eles também escrevem que tem uma multiplicação por "random offset", e de fato os valores estariam muito grandes, então há um offset a mais de fato
        ax.autoscale()
        formatter11=ScalarFormatterClass1()
        formatter11.set_scientific(True)
        formatter11.set_powerlimits((0,0))
        ax.yaxis.set_major_formatter(formatter11)
        #plt.show()
        pp.savefig()
        plt.close(fig)
    pp.close()
       
    pp = PdfPages(f"./plots_3d_filtered_spectras/{sn}.pdf")
    fig = plt.figure(figsize=(15,12))
    ax = plt.axes(projection='3d')
    for p in filtered_reg_data[sn].keys():
        X, Y = np.meshgrid(p,filtered_reg_data[sn][p]["wave"].values)
        Z=np.array(filtered_reg_data[sn][p]["flux"].values).reshape(Y.shape)
        ax.plot_wireframe(X,Y,Z,color="black",lw=2)
        ax.set_xlabel(r"Phase (days)")
        ax.set_ylabel(r"Wavelength ($\AA$)")
        ax.set_zlabel(r"Flux (erg/s/cm$^2/\AA$ $\times$ offset)")
        ax.yaxis.labelpad=20
        ax.xaxis.labelpad=15
        ax.zaxis.labelpad=15
    #plt.show()
    pp.savefig()
    plt.close(fig)
    pp.close()

100%|█████████████████████████████████████████| 171/171 [05:07<00:00,  1.80s/it]


## Saving filtered spectras

In [11]:
for sn in filtered_reg_data.keys():
    f=bz2.BZ2File(f'./filtered_spectras/{sn}.pkl', 'wb')
    pickle.dump(filtered_reg_data[sn], f)
    f.close()